In [25]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)


True

In [3]:
import pathlib
import textwrap
import google.generativeai as genai
import tqdm as notebook_tqdm
from IPython.display import display
from IPython.display import Markdown

In [4]:
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
GPT4_API_KEY = os.environ.get("GPT4_API_KEY")

In [5]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [6]:
genai.configure(api_key=GOOGLE_API_KEY)

### Loading the PDF doc

In [7]:
from langchain_community.document_loaders import PyPDFLoader

In [8]:
loader = PyPDFLoader("/Users/uzair/tech_fusion/vaswani_attention_v2.pdf")
pages = loader.load_and_split()

In [9]:
pages[0]


Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing w

In [10]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings

### Converting the splitted documents to Embeddings

In [11]:
embeddings = GPT4AllEmbeddings()

In [12]:
db = FAISS.from_documents(pages, embeddings)


In [13]:
docs = db.similarity_search("Applications of Attention in our Model", k=2)

In [14]:
docs

[Document(page_content='Attention Visualizations\nInput-Input Layer5\nIt\nis\nin\nthis\nspirit\nthat\na\nmajority\nof\nAmerican\ngovernments\nhave\npassed\nnew\nlaws\nsince\n2009\nmaking\nthe\nregistration\nor\nvoting\nprocess\nmore\ndifficult\n.\n<EOS>\n<pad>\n<pad>\n<pad>\n<pad>\n<pad>\n<pad>\nIt\nis\nin\nthis\nspirit\nthat\na\nmajority\nof\nAmerican\ngovernments\nhave\npassed\nnew\nlaws\nsince\n2009\nmaking\nthe\nregistration\nor\nvoting\nprocess\nmore\ndifficult\n.\n<EOS>\n<pad>\n<pad>\n<pad>\n<pad>\n<pad>\n<pad>\nFigure 3: An example of the attention mechanism following long-distance dependencies in the\nencoder self-attention in layer 5 of 6. Many of the attention heads attend to a distant dependency of\nthe verb ‘making’, completing the phrase ‘making...more difficult’. Attentions here shown only for\nthe word ‘making’. Different colors represent different heads. Best viewed in color.\n13', metadata={'source': '/Users/uzair/tech_fusion/vaswani_attention_v2.pdf', 'page': 12}),
 D

In [15]:
retriever = db.as_retriever()


In [16]:
docs = retriever.get_relevant_documents("Applications of Attention in our Mode")


/Users/uzair/miniconda3/envs/fusion/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [17]:
docs

[Document(page_content='Attention Visualizations\nInput-Input Layer5\nIt\nis\nin\nthis\nspirit\nthat\na\nmajority\nof\nAmerican\ngovernments\nhave\npassed\nnew\nlaws\nsince\n2009\nmaking\nthe\nregistration\nor\nvoting\nprocess\nmore\ndifficult\n.\n<EOS>\n<pad>\n<pad>\n<pad>\n<pad>\n<pad>\n<pad>\nIt\nis\nin\nthis\nspirit\nthat\na\nmajority\nof\nAmerican\ngovernments\nhave\npassed\nnew\nlaws\nsince\n2009\nmaking\nthe\nregistration\nor\nvoting\nprocess\nmore\ndifficult\n.\n<EOS>\n<pad>\n<pad>\n<pad>\n<pad>\n<pad>\n<pad>\nFigure 3: An example of the attention mechanism following long-distance dependencies in the\nencoder self-attention in layer 5 of 6. Many of the attention heads attend to a distant dependency of\nthe verb ‘making’, completing the phrase ‘making...more difficult’. Attentions here shown only for\nthe word ‘making’. Different colors represent different heads. Best viewed in color.\n13', metadata={'source': '/Users/uzair/tech_fusion/vaswani_attention_v2.pdf', 'page': 12}),
 D

In [18]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [19]:
llm = OpenAI(openai_api_key=GPT4_API_KEY,temperature=0)

/Users/uzair/miniconda3/envs/fusion/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [20]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

### 1# Applications of Attention in our Model

In [21]:
query = "Fetch the extact content of Applications of Attention in our Model"
result = qa_chain({"query": query})
print(result['result'])

/Users/uzair/miniconda3/envs/fusion/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




The Transformer uses multi-head attention in three different ways:

•In "encoder-decoder attention" layers, the queries come from the previous decoder layer,
and the memory keys and values come from the output of the encoder. This allows every
position in the decoder to attend over all positions in the input sequence. This mimics the
typical encoder-decoder attention mechanisms in sequence-to-sequence models such as
[38, 2, 9].

•The encoder contains self-attention layers. In a self-attention layer all of the keys, values
and queries come from the same place, in this case, the output of the previous layer in the
encoder. Each position in the encoder can attend to all positions in the previous layer of the
encoder.

•Similarly, self-attention layers in the decoder allow each position in the decoder to attend to
all positions in the decoder up to and including that position. We need to prevent leftward
information flow in the decoder to preserve the auto-regressive property. We impleme

### 2# Content Under Training

In [22]:
query = "Extract the entire extract content under the training regime for our models"
result = qa_chain({"query": query})
print(result['result'])


5 Training
This section describes the training regime for our models.
5.1 Training Data and Batching
We trained on the standard WMT 2014 English-German dataset consisting of about 4.5 million
sentence pairs. Sentences were encoded using byte-pair encoding [ 3], which has a shared source-
target vocabulary of about 37000 tokens. For English-French, we used the significantly larger WMT
2014 English-French dataset consisting of 36M sentences and split tokens into a 32000 word-piece
vocabulary [ 38]. Sentence pairs were batched together by approximate sequence length. Each training
batch contained a set of sentence pairs containing approximately 25000 source tokens and 25000
target tokens.
5.2 Hardware and Schedule
We trained our models on one machine with 8 NVIDIA P100 GPUs. For our base models using
the hyperparameters described throughout the paper, each training step took about 0.4 seconds. We
trained the base models for a total of 100,000 steps or 12 hours. For our big models,(descri

### 3# Extract Table 2 &  make a csv from the prompt.

In [23]:
query = "Extract the table no 2 from the document in csv form"
result = qa_chain({"query": query})
table_res  = result['result']
print(table_res)



Model, BLEU, Training Cost (FLOPs)
ByteNet [18], 23.75, N/A
Deep-Att + PosUnk [39], 39.2, 1.0·1020
GNMT + RL [38], 24.6, 39.92, 2.3·10191.4·1020
ConvS2S [9], 25.16, 40.46, 9.6·10181.5·1020
MoE [32], 26.03, 40.56, 2.0·10191.2·1020
Deep-Att + PosUnk Ensemble [39], 40.4, 8.0·1020
GNMT + RL Ensemble [38], 26.30, 41.16, 1.8·10201.1·1021
ConvS2S Ensemble [9], 26.36, 41.29, 7.7·10191.2·1021
Transformer (base model), 27.3, 38.1, 3.3·1018
Transformer (big), 28.4, 


#### Note : I have explored the alternative solution in other notebook for problem statement-3

### 4# extract the content from the pdf as 1 or 2 lines, for all the models (first five) mentioned in table 2, assigned with numerical values.

In [24]:
query = "extract the numerical values written as [] in model column in table 2"
result = qa_chain({"query": query})
table_res  = result['result']
print(table_res)



ModelBLEU Training Cost (FLOPs)
EN-DE EN-FR EN-DE EN-FR
ByteNet [18] 23.75
Deep-Att + PosUnk [39] 39.2 1.0·1020
GNMT + RL [38] 24.6 39.92 2.3·10191.4·1020
ConvS2S [9] 25.16 40.46 9.6·10181.5·1020
MoE [32] 26.03 40.56 2.0·10191.2·1020
Deep-Att + PosUnk Ensemble [39] 40.4 8.0·1020
GNMT + RL Ensemble [38] 26.30 41.16 1.8·10201.1·1021
ConvS2S Ensemble [9] 26.36 41.29 7.7·10191.2·1021
Transformer (base model) 27.3 38.1 3.3·1018
Transformer (big) 28.4 41.8 2.3
